<a href="https://colab.research.google.com/github/dmburns1729/Class-Files/blob/main/9_12_CodeAlong_Diamonds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Steps
1. import all libraries
2. Load the dataset
3. make copy of df
5. check about dataset info
6. df.isna().sum().sum()
7. check for duplicates and drop the duplicates - df.duplicated().sum()
8. check for incosistacneis
9. Split the dataset-train/test
10. column selection - separate the categorical and numerical one data
11. handling missing values using imputer for categorical and numerical dataset
12. justify the strategy used in code like why are you using mean, median or mode.
13. Make pipleines
14. use column transformers for scaling , ohe, for the categorical and numerical data.
15.fit the data to train and transform train/test both  


In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, \
OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display='diagram')

# Task:
Predict the sale price of a diamond based on measurements of the diamond

carat
weight of the diamond

cut
quality of the cut (Fair, Good, Very Good, Premium, Ideal)

color
diamond colour, from J (worst) to D (best)

clarity
a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))

x
length in mm

y
width in mm

z
depth in mm

table

width of top of diamond relative to widest point

# Steps:

1. Load the Data
2. Check for duplicates, missing values, errors
3. Split the data (X, y and train, test)
4. Instantiate the Transformers and column selectors
5. Create numeric preprocessing pipeline and categorical preprocessing pipeline
6. Combine pipelines using column transformer
7. Fit preprocessor on X_train
8. Transform both X_train and X_test
9. (Optional) convert processed X_train back to a dataframe.

# Load the Data

In [ ]:
# Load in the data

df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTpBcQrzjkPeQ4W_sZuOa45qqxLBgtr_wzistIB3a7gFUJ1AlBhloV5g9IxV5DFnd-GPktXDNprGOUh/pub?gid=88799590&single=true&output=csv')

In [ ]:
# Take a look at the first five rows of the dataframe
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326.0,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326.0,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327.0,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334.0,4.20,4.23,2.63
4,5,NaN,Good,J,SI2,63.3,58.0,335.0,4.34,4.35,2.75


We will make a copy of original df to avoid any manipulations

In [ ]:
df_ml = df.copy()

In [ ]:
df_ml.drop('Unnamed: 0', axis=1, inplace=True)

# Check for Duplicated, Missing, or Erroneous Data

In [ ]:
df_ml.isna().sum()

carat      20
cut        21
color       8
clarity     4
depth       6
table       5
price       6
x           5
y           6
z           6
dtype: int64

In [ ]:
# Check to see if there are any duplicates
df_ml.duplicated().sum()

145

In [ ]:
df_ml.drop_duplicates(inplace=True)

In [ ]:
# Look at the info from the data
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53795 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53775 non-null  float64
 1   cut      53774 non-null  object 
 2   color    53787 non-null  object 
 3   clarity  53791 non-null  object 
 4   depth    53789 non-null  float64
 5   table    53790 non-null  float64
 6   price    53789 non-null  float64
 7   x        53790 non-null  float64
 8   y        53789 non-null  float64
 9   z        53789 non-null  float64
dtypes: float64(7), object(3)
memory usage: 4.5+ MB


In [ ]:
df_ml.describe(include='all')

,carat,cut,color,clarity,depth,table,price,x,y,z
count,53775.000000,53774,53787,53791,53789.000000,53790.000000,53789.000000,53790.000000,53789.000000,53789.000000
unique,NaN,5,7,8,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Ideal,G,SI1,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,21482,11261,13031,NaN,NaN,NaN,NaN,NaN,NaN
mean,0.797843,NaN,NaN,NaN,61.748088,57.458130,3933.306178,5.731268,5.734716,3.538758
std,0.473441,NaN,NaN,NaN,1.429968,2.233664,3988.219384,1.120706,1.141239,0.705048
min,0.200000,NaN,NaN,NaN,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,NaN,NaN,NaN,61.000000,56.000000,951.000000,4.710000,4.720000,2.910000
50%,0.700000,NaN,NaN,NaN,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,NaN,NaN,NaN,62.500000,59.000000,5327.000000,6.540000,6.540000,4.030000


# Drop Rows with No Price

In [ ]:
df_ml.dropna(subset=['price'], inplace=True)

How many rows would we lose if we just dropped the rows with missing values?

In [ ]:
df_ml.dropna().shape[0] / df.shape[0]

0.9959213941416388

# Split the Data (Validation Split)

In [ ]:
# split X and y, you are predicting price
X = df_ml.drop(columns=['price'])
y = df_ml['price']

# split training and test
# set random_state to 42 for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30 , random_state=42)

In [ ]:
# select only object columns
categoricals = X_train.select_dtypes(include='object')

# check value counts for each object column
for col in categoricals.columns:
  print(col)
  print(categoricals[col].value_counts(), '\n')

cut
Ideal        14977
Premium       9595
Very Good     8540
Good          3402
Fair          1128
Name: cut, dtype: int64 

color
G    7943
E    6791
F    6672
H    5775
D    4696
I    3784
J    1986
Name: color, dtype: int64 

clarity
SI1     9167
VS2     8570
SI2     6347
VS1     5693
VVS2    3551
VVS1    2574
IF      1238
I1       511
Name: clarity, dtype: int64 



# Instantiate Transformers

In [ ]:
#instantiate the StandardScaler, OneHotEncoder, OrdinalEncoder and Imputers
scaler1 = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')

#this is showing for learning purpose but we are not using it here.
missing_imputer = SimpleImputer(strategy='constant', fill_value='missing')

# Ordinal Encoder

The `cut` column was pretty clear in how to order them, but the clarity column took some research.  [Selecting a Diamond](https://selectingadiamond.com/diamond-clarity/)

![Diamond Clarity Chart](https://selectingadiamond.com/wp-content/uploads/2019/10/Diamond-Clarity.jpg)

In [ ]:
#create a list of ordinal labels, in order of least to most

cut_labels = ['Fair','Good','Very Good', 'Premium', 'Ideal']
clarity_labels = ['I1', 'SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

#combine the ordered list in the order that the columns appear
ordered_labels = [cut_labels, clarity_labels]

#handle_unknown is 'error' by default.  That's a good place to start
#but it may cause problems in a production model.
ordinal = OrdinalEncoder(categories = ordered_labels)

# Create a Pipeline Numeric Data and a Pipeline for Categorical Data

In [ ]:
# Setup the pipelines for the numeric and categorical data

num_pipeline = make_pipeline(mean_imputer, scaler1)
ord_pipeline = make_pipeline(freq_imputer, ordinal)
nom_pipeline = make_pipeline(missing_imputer, ohe)

# Create Tuples to Pair Pipelines with Columns

In [ ]:
# Create column lists for objects and a number selector
ordinal_cols = ['cut', 'clarity']
nominal_cols = ['color']

num_selector = make_column_selector(dtype_include='number')

# Setup the tuples to pair the processors with the make column selectors

numeric_tuple = (num_pipeline, num_selector)
ordinal_tuple = (ord_pipeline, ordinal_cols)
nominal_tuple = (nom_pipeline, nominal_cols)

# Create Column Transformer to Apply Different Preprocessing to Different Columns

In [ ]:
# Instantiate the make column transformer

preprocessor = make_column_transformer(ordinal_tuple,
                                          numeric_tuple,
                                          nominal_tuple,
                                          remainder='drop')

# Fit the Column Transformer on the Training Data

In [ ]:
# Fit the column transformer on the X_train

preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']]))]),
                                 ['cut', 'clarity']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f6b11ec06d0>),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['color'])])

# Transform Both Training and Testing Data

In [ ]:
# Transform the X_train and the X_test

X_train_transformed = preprocessor.transform(X_train)

X_test_transformed = preprocessor.transform(X_test)

In [ ]:
X_train_transformed.shape

(37652, 16)

# Check the Result

In [ ]:
X_train_transformed[:5]

array([[3.        , 2.        , 0.29313929, 0.55277778, 0.48484848,
        0.69832402, 0.2336478 , 0.67191977, 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [2.        , 3.        , 0.07068607, 0.55833333, 0.45454545,
        0.48510242, 0.16132075, 0.46704871, 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [4.        , 6.        , 0.05613306, 0.51666667, 0.39393939,
        0.46554935, 0.15786164, 0.44269341, 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [3.        , 1.        , 0.17879418, 0.49722222, 0.51515152,
        0.60986965, 0.20503145, 0.57020057, 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [1.        , 2.        , 0.04573805, 0.56388889, 0.39393939,
        0.44227188, 0.15062893, 

In [ ]:
np.isnan(X_train_transformed).sum()

0

# Problem to Solve:
1. Should we scale ordinal data?



# (Optional) Preview of Next Week!  Modeling!

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
lin_reg = LinearRegression()

lin_reg_pipe = make_pipeline(preprocessor, lin_reg)

lin_reg_pipe.fit(X_train, y_train)

train_predictions = lin_reg_pipe.predict(X_train)
test_predictions = lin_reg_pipe.predict(X_test)

In [ ]:
train_error = mean_absolute_error(y_train, train_predictions)
test_error = mean_absolute_error(y_test, test_predictions)

print(f'The models average error on the training set is {train_error}')
print(f'the models average error on the testing set is {test_error}')

The models average error on the training set is 791.3396633768033
the models average error on the testing set is 781.774189362006
